In [158]:
import pandas as pd
import nltk
import re
import string
from textblob import TextBlob
import numpy as np
import gensim
import os

In [111]:
!pip uninstall tflearn
!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing

Found existing installation: tflearn 0.5.0
Uninstalling tflearn-0.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tflearn-0.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tflearn/*
Proceed (Y/n)? y
  Successfully uninstalled tflearn-0.5.0
  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-mw37fnin
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-mw37fnin
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=130659 sha

In [106]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [107]:
import json
with open('intents.json') as file:
  dictionary = json.load(file)

print(dictionary['intents'])

[{'tag': 'greeting', 'patterns': ['Hello', 'Hi', 'Hey', 'Greetings'], 'responses': ['Hello! How can I assist you today?', 'Hi there! How may I help you?', 'Hey! Welcome. How can I be of service?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Goodbye', 'Bye', 'See you later', 'Farewell'], 'responses': ['Goodbye! If you have any more questions, feel free to ask.', 'Bye! Have a great day!', 'See you later! Safe travels.'], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thank you', 'Thanks', 'Appreciate it'], 'responses': ["You're welcome! If you need anything else, feel free to ask.", "No problem! Let me know if there's anything else I can do for you.", 'Glad I could help! If you have more questions, just ask.'], 'context_set': ''}, {'tag': 'help', 'patterns': ['Help', 'Assistance', 'Can you help me?'], 'responses': ["Of course! I'm here to assist you. What do you need help with?", 'Certainly! What can I help you with today?', "Sure, I'm here to help. What do you require ass

In [108]:
data = pd.DataFrame(dictionary['intents'])
def toString(arr):
  info = ""
  for elements in arr:
    info += elements +" "
  return info

data['patterns']=data['patterns'].apply(toString)
print(data)

                   tag                                           patterns  \
0             greeting                            Hello Hi Hey Greetings    
1              goodbye                Goodbye Bye See you later Farewell    
2               thanks                    Thank you Thanks Appreciate it    
3                 help                  Help Assistance Can you help me?    
4           book_hotel  Book a hotel Reserve a room Hotel booking I wa...   
5   cancel_reservation  Cancel reservation I want to cancel my booking...   
6   check_availability  Check room availability Are there any rooms av...   
7   modify_reservation  Modify reservation Change booking details Upda...   
8     get_confirmation  Get booking confirmation Can I have my reserva...   
9         pricing_info  How much does a room cost? Tell me about your ...   
10           amenities  Tell me about hotel amenities What facilities ...   
11            feedback  Leave feedback Provide a review I want to shar...   

In [109]:
from nltk.corpus import stopwords

def remove_punc(text):
  exclude = set(string.punctuation)
  for char in exclude:
    text = text.replace(char,'')
  return text

def stop(text):
  new = []
  sw = stopwords.words('english')
  for words in text.split():
    if words in sw:
      new.append('')
    else:
      new.append(words)
  x = new[:]
  new.clear()
  return " ".join(x)


data['patterns']=data['patterns'].str.lower()
data['patterns']=data['patterns'].apply(remove_punc)
data['patterns']=data['patterns'].apply(stop)

print(data['patterns'])

0                                hello hi hey greetings
1                       goodbye bye see  later farewell
2                             thank  thanks appreciate 
3                               help assistance   help 
4     book  hotel reserve  room hotel booking  want ...
5     cancel reservation  want  cancel  booking canc...
6     check room availability    rooms available  ro...
7     modify reservation change booking details upda...
8     get booking confirmation     reservation detai...
9       much   room cost tell    prices    rate   night
10    tell   hotel amenities  facilities    list hot...
11    leave feedback provide  review  want  share  e...
Name: patterns, dtype: object


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,1))
new = cv.fit_transform(data['patterns']).toarray()

In [133]:
import tflearn
import tensorflow as tf
output = []
for tag in data['tag']:
  row=[0 for _ in range(len(data['tag']))]
  row[data.index[data['tag']==tag].tolist()[0]]=1
  output.append(row)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape = [None,len(new[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [104]:
print(len(new[0]))
print(len(output[0]))

50
12


In [134]:
model.fit(new,output,n_epoch = 100,batch_size = 1,show_metric = True)

Training Step: 1199  | total loss: 0.60687 | time: 0.072s
| Adam | epoch: 100 | loss: 0.60687 - acc: 0.7507 -- iter: 11/12
Training Step: 1200  | total loss: 0.58937 | time: 0.087s
| Adam | epoch: 100 | loss: 0.58937 - acc: 0.7757 -- iter: 12/12
--


In [157]:
import random
message = "Rooms available?"
test = cv.transform([message])
results = model.predict(test.toarray())
tag = data['tag'][np.argmax(results)]
resp = data.index[data['tag']==tag].tolist()[0]
print(random.choice(data['responses'][resp]))

Certainly! To assist you with room availability, may I know the check-in and check-out dates?
